# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [67]:
import pandas as pd
import numpy as np
import math
import json


# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [65]:
print(transcript)

                  event                            person  time  \
0        offer received  78afa995795e4d85b5d9ceeca43f5fef     0   
1        offer received  a03223e636434f42ac4c3df47e8bac43     0   
2        offer received  e2127556f4f64592b11af22de27a7932     0   
3        offer received  8ec6ce2a7e7949b1bf142def7d0e0586     0   
4        offer received  68617ca6246f4fbc85e91a2a49552598     0   
5        offer received  389bc3fa690240e798340f5a15918d5c     0   
6        offer received  c4863c7985cf408faee930f111475da3     0   
7        offer received  2eeac8d8feae4a8cad5a6af0499a211d     0   
8        offer received  aa4862eba776480b8bb9c68455b8c2e1     0   
9        offer received  31dda685af34476cad5bc968bdb01c53     0   
10       offer received  744d603ef08c4f33af5a61c8c7628d1c     0   
11       offer received  3d02345581554e81b7b289ab5e288078     0   
12       offer received  4b0da7e80e5945209a1fdddfe813dbe0     0   
13       offer received  c27e0d6ab72c455a8bb66d980963de60     

# Cleaning the Data

In [68]:
# a row of data for the ML model is going to be everything we know about an 'offer recieved': offer type, profile of offeree
# $ spent by person within timespan of offer

#first, break up offers from transactions
offers = transcript[transcript['event'] != 'transaction']
transactions = transcript[transcript['event'] == 'transaction']

#pull data out of the dictionary object
offers['offer_id'] = pd.Series([i.get('offer id') for i in offers['value']])
transactions['amount'] = [float(i.get('amount')) for i in transactions['value']]

#clean up profile date, need to make the member since attribute and integer (days from min date)
profile['member_date'] = pd.Series([pd.to_datetime(str(i)) for i in profile['became_member_on']])
min_date = min(profile['member_date'])
profile['member_date_int'] = pd.Series([i - min_date for i in profile['member_date']]).dt.days

#make dummy variables for m/f gender. baseline in gender = 'none'
profile = pd.concat([profile, pd.get_dummies(profile['gender'])], axis=1)

#impute values for income, add impute_income flag
mean_income = round(np.mean(profile['income'][profile['income'].notnull()]))

income, income_flag = [],[]
for index, i in enumerate(profile['income'].isnull()):
    if i:
        income.append(mean_income)
        income_flag.append(1)
    else:
        income.append(profile['income'][index])
        income_flag.append(0)
        
profile['income'] = pd.Series(income)
profile['income_impute_flag'] = pd.Series(income_flag)

#remove redundant columns
profile = profile.drop(['became_member_on','member_date','gender'],axis=1)

#merge cleaned up dataframes
offers = offers.merge(portfolio,how='left',left_on='offer_id', right_on='id')
offers = offers.merge(profile,how='left',left_on='person', right_on='id')

offers = offers.drop(['id_x', 'id_y','value','person','offer_id'], axis=1)
print(offers.head())


C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\duran\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


            event  time                      channels  difficulty  duration  \
0  offer received     0          [web, email, mobile]         5.0       7.0   
1  offer received     0                  [web, email]        20.0      10.0   
2  offer received     0          [web, email, mobile]        10.0       7.0   
3  offer received     0  [web, email, mobile, social]        10.0      10.0   
4  offer received     0  [web, email, mobile, social]        10.0       5.0   

  offer_type  reward  age    income  member_date_int  F  M  O  \
0       bogo     5.0   75  100000.0             1380  1  0  0   
1   discount     5.0  118   65405.0             1467  0  0  0   
2   discount     2.0   68   70000.0             1732  0  1  0   
3   discount     2.0  118   65405.0             1519  0  0  0   
4       bogo    10.0  118   65405.0             1526  0  0  0   

   income_impute_flag  
0                   0  
1                   1  
2                   0  
3                   1  
4             